In [1]:
import traitlets, requests, ipywidgets, pandas, jinja2, IPython, odo, multipledispatch, blaze, logging, http, sys
from coffeetools import coffee
%reload_ext autoreload
%autoreload 2
get_ipython = ipywidgets.get_ipython

In [51]:
@odo.resource.register( 'gh://.*' )
def github__shorthand_resource( path, *args, **kwargs ):
    """Get user or repository level information."""
    __route__ = 'gh://'
    user = path.split( __route__ )[1].split('/')
    user, repo = user if len(user) == 2 else [user[0],'']
    method = 'repos' if repo else 'users'    
    return jinja2.Template( 'https://api.github.com/{{method}}/{{user}}/{{repo}}').render(**locals()).rstrip('/')

@multipledispatch.dispatch( str, dict, requests.sessions.Session )
def github_request( url, params = {}, session = requests.Session() ):
    """
    Make the URL when a url is passed to the function
    
            req, resp = github_request('tonyfast', {}, requests.Session() )
    """
    if not url.startswith('http'):
        url = odo.resource( 'gh://' + url )
    request = requests.Request('GET', url=url, params=params.copy(), headers={'User-Agent': 'tonyfast'} )
    request.params = append_auth( request.params )
    return request, session.send( request.prepare() )

@multipledispatch.dispatch( requests.models.Request, requests.models.Response )
def github_request( requests, response ):
    del requests.params['access_token']
    return requests.prepare(), response.json()

In [53]:
class Projects( traitlets.HasTraits ):
    current = traitlets.List([])
    access_token = ipywidgets.Text('asdfasdf')
    username = ipywidgets.Text('tonyfast')
    columns = ipywidgets.SelectMultiple( options=['<Empty>'] )
    row = ipywidgets.IntSlider( description='Row Index', min=0, max=0 )
    session = requests.Session()
    catalog = {}   
    
    def __init__( self, projects = [], *args, **kwargs ):
        self.env = jinja2.Environment( loader = jinja2.FileSystemLoader('_layouts') )
        for project in projects:
            self.add_project( project )
                
    @traitlets.observe('current')
    def _update_projects(self, change, *args, **kwargs):
        for project in change['new']:            
            if not project in change['old']:
                self._make_request( project, alias=project )

    def _make_request( self, project_or_url, params = {}, alias=None ):     
        recorded = self._record( *github_request( project_or_url, params, self.session ) )
        if alias:
            self.catalog[alias] = recorded
        
    def _record( self, request, response ):
        url = request.prepare().url
        self.catalog[url] = {
            'response': response,
            'time': str(pandas.datetime.now()),
            'params': request.params.copy(),
            'request': request,
            'url': url
        } 
        return self.catalog[url] 

    def more( self, project_or_url ):
        params = {}
        self._make_request( project_or_url, params, self.session )
        
    def add_project( self, project ):
        tmp = self.current.copy()
        tmp.append(project)
        self.current = tmp

In [54]:
projects = Projects(['jupyter/notebook'])

In [55]:
projects.add_project('bokeh/bokeh')

In [56]:
projects.catalog

{'bokeh/bokeh': {'params': {},
  'request': <Request [GET]>,
  'response': <Response [200]>,
  'time': '2016-02-17 11:59:20.075391',
  'url': 'https://api.github.com/repos/bokeh/bokeh'},
 'https://api.github.com/repos/bokeh/bokeh': {'params': {},
  'request': <Request [GET]>,
  'response': <Response [200]>,
  'time': '2016-02-17 11:59:20.075391',
  'url': 'https://api.github.com/repos/bokeh/bokeh'},
 'https://api.github.com/repos/jupyter/notebook': {'params': {},
  'request': <Request [GET]>,
  'response': <Response [200]>,
  'time': '2016-02-17 11:59:19.213879',
  'url': 'https://api.github.com/repos/jupyter/notebook'},
 'jupyter/notebook': {'params': {},
  'request': <Request [GET]>,
  'response': <Response [200]>,
  'time': '2016-02-17 11:59:19.213879',
  'url': 'https://api.github.com/repos/jupyter/notebook'}}

In [50]:
%%file _layouts/table.html
<table id="catalog" class="table"></table>

Overwriting _layouts/table.html


In [51]:
%%file _layouts/update_table.coffee
data = {{df}}
table = d3.select '#catalog'
header = table.selectAll 'tr.header'
    .data [ '', data['columns'] ]
header.enter().append('tr').classed 'header', true
header.each (d)->
    cells = d3.select(@).selectAll('th').data d
    cells.enter().append 'th'
    cells.text (d)-> d
    cells.exit().remove()
header.exit().remove()
value = table.selectAll 'tr.value'
    .data data['data']
value.enter().append('tr').classed 'value', true
value.each (d)->
    cells = d3.select @
        .selectAll 'td'
        .data d
    cells.enter().append 'td'
    cells.text (d)-> JSON.stringify d
    cells.exit().remove()  
value.exit().remove()

Overwriting _layouts/update_table.coffee
